# Multiplayer Metaflow

## Sharing

Data scientist's on the same Metaflow deployment have read access to each other's Metaflow results. 
This is useful in a project because you can make incremental progress on each other results, without reinventing the wheel.
You can also compare approaches, results, and split work in a seamless way.

Two concepts and sharing features that you should understand to benefit most from Metaflow in a team setting, are namespaces and projects.

## Namespaces

[Namespaces](https://docs.metaflow.org/scaling/tagging#namespaces) help you to keep results organized. 
A flow runs in a namespace, and the Metaflow Client API retrieves results for you based on the active namespace.
Namespaces are not about security or access control, but for organizing results by the users who produce and analyze them.

In [ ]:
from metaflow import namespace, Metaflow

In [ ]:
namespace('user:sandbox')
Metaflow().flows

In [ ]:
# Expect: the same result in this global (None) namespace as the user:sandbox namespace, 
    # because your sandbox user is the only user on this Metaflow deployment.
namespace(None)
Metaflow().flows

## The production namespace
There is a special namespace designated production. Production namespaces are important for a variety of reasons. For example, it helps you add authorization keys to the process of deploying to a valuable Flow's production namespace. 

To get started in production, find your `Branch_Cloud_Flow` Flow from the previous section and run this command with the file path changed to wherever you place the flow.

In [ ]:
# Watch the file path if you run in the terminal! 

# This will take a minute or two the first time it has to build the conda environment.
# Find the namespace this flow is deployed in. Is it production? 🧐

! python ../flows/cloud/tree_branch_cloud_flow_deploy.py --environment=conda argo-workflows create

In the output, you will see a line like `namespace("user:sandbox")` or `namespace("production:branch_cloud_flow-0-bkst")`. 
Copy that line, paste it in the first line of the next cell and run it. Then 

In [ ]:
### REPLACE THIS LINE WITH YOUR... 
# namespace("production:branch_cloud_flow-X-XXXX")

In [ ]:
# run this cell to print your Argo URL
import json
res = json.load(open("/home/workspace/.metaflowconfig/config.json"))
print("Go to this URL:", res['SANDBOX_VSCODE_URL'].replace('vs', 'argo'), "and click on the second top left tab 'Workflow Templates'. Then come back here and trigger the flow.")

In [ ]:
! python ../flows/cloud/tree_branch_cloud_flow_deploy.py --environment=conda argo-workflows trigger

In [ ]:
# This cell will return an empty list for a minute or two after Argo UI shows the flow running.
Metaflow().flows
# Do you see a different set of flows than what you saw above with the global or user namespace. 

## Projects
The [`@project decorator`](https://docs.metaflow.org/production/coordinating-larger-metaflow-projects) is for production use cases. 

It makes available three classes of namespaces that will affect the behavior of a production deployment:
1. `user` is the default. It will deploy to a user-specific, private namespace. Use it for testing production deployments.
2. `test` denotes custom branches that can be shared amongst multiple users. Use it for deploying experimental versions that can run in parallel with production. Deploy custom branches with `--branch foo`.
3. `prod` denotes the global production namespace. Use it for deploying the official production version of the project. Deploy to production with `--production`. For multiple production variants, deploy custom branches with `--production --branch foo`.

You don't need to remember these, but they are useful to revisit once you have thought more about the requirements of your ML project.
For example, later in this week you will consider how to deploy multiple production variants to score a champion/challenger model on production traffic.

### Motivation
Consider the situation after you deploy `Branch_Cloud_Flow` to Argo that is running in production. 
The next time you deploy `Branch_Cloud_Flow` by running the same command you just did a few cells ago,
```sh
python ../flows/cloud/tree_branch_cloud_flow_deploy.py --environment=conda argo-workflows create
```
it will overwrite the production `Branch_Cloud_Flow` flow. Clearly, we want more optionality to run experiments.

What do you do when the workflow starts performing well, and multiple people want to test their own production deployments without interfering with yours; or if, as a single developer, you want to experiment with multiple independent deployments of your workflow? How do you create a new workflow without overwriting the existing one? 

### Going to --production

Metaflow's `@project` decorator makes it easy to specifiy the production namespace. You can use this to deploy workflows in different namespaces, and specify a dedicated production branch that is not for anything experimental.

Go to your `flows/cloud/tree_branch_cloud_flow_deploy.py` implementation, and add the `@project` decorator to your flow. 

In [ ]:
! python ../flows/cloud/tree_branch_cloud_flow_deploy.py --environment=conda --production argo-workflows create

In [ ]:
! python ../flows/cloud/tree_branch_cloud_flow_deploy.py --environment=conda --production argo-workflows trigger